In [ ]:
%matplotlib inline
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,6)

### Implicit Multithreading in NumPy

#### A Matrix Operation